--- Day 3: Crossed Wires ---

The gravity assist was successful, and you're well on your way to the Venus refuelling station. During the rush back on Earth, the fuel management system wasn't completely installed, so that's next on the priority list.

Opening the front panel reveals a jumble of wires. Specifically, two wires are connected to a central port and extend outward on a grid. You trace the path each wire takes as it leaves the central port, one wire per line of text (your puzzle input).

The wires twist and turn, but the two wires occasionally cross paths. To fix the circuit, you need to find the intersection point closest to the central port. Because the wires are on a grid, use the Manhattan distance for this measurement. While the wires do technically cross right at the central port where they both start, this point does not count, nor does a wire count as crossing with itself.

For example, if the first wire's path is R8,U5,L5,D3, then starting from the central port (o), it goes right 8, up 5, left 5, and finally down 3:

    ...........
    ...........
    ...........
    ....+----+.
    ....|....|.
    ....|....|.
    ....|....|.
    .........|.
    .o-------+.
    ...........

Then, if the second wire's path is U7,R6,D4,L4, it goes up 7, right 6, down 4, and left 4:

    ...........
    .+-----+...
    .|.....|...
    .|..+--X-+.
    .|..|..|.|.
    .|.-X--+.|.
    .|..|....|.
    .|.......|.
    .o-------+.
    ...........

These wires cross at two locations (marked X), but the lower-left one is closer to the central port: its distance is 3 + 3 = 6.

Here are a few more examples:

    R75,D30,R83,U83,L12,D49,R71,U7,L72
    U62,R66,U55,R34,D71,R55,D58,R83 = distance 159
    R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
    U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = distance 135

What is the Manhattan distance from the central port to the closest intersection?


In [1]:
sample_0 = '''R8,U5,L5,D3
    U7,R6,D4,L4'''

sample_1 = '''R75,D30,R83,U83,L12,D49,R71,U7,L72
    U62,R66,U55,R34,D71,R55,D58,R83'''

sample_2 = '''R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
    U98,R91,D20,R16,D67,R40,U7,R15,U6,R7'''

input_03 = [line.strip() for line in open('03_input.txt')]

In [2]:
def wander(path):
    '''Fill in the steps taken on a journey and return a set of visited coordinates.'''
    # Rather than tuples, use Walt Mankowski's method of using complex numbers in the set
    # Real numbers are the x axis, imaginary are the y
    # See https://python-reference.readthedocs.io/en/latest/docs/complex/ for help

    direction={
        'R': +1,
        'L': -1,
        'U': +1j,
        'D': -1j
    }
    journey = set()
    pos = 0+0j
    for vector in path:
        facing,dist = vector[0],int(vector[1:])
        #print(facing,dist)
        for n in range(0,dist):
            journey.add(pos + n*direction[facing])
        pos += dist*direction[facing]
    return journey

In [3]:
path = [p.split(',') for p in [l.strip() for l in sample_2.split('\n')] ]
print(path[1])

['U98', 'R91', 'D20', 'R16', 'D67', 'R40', 'U7', 'R15', 'U6', 'R7']


In [4]:
path = [p.split(',') for p in input_03]

j1 = wander(path[0])
j2 = wander(path[1])
print(j1.intersection(j2))
blocks = [abs(x.real) + abs(x.imag) for x in j1.intersection(j2)  ]
print(blocks)
print(min( [x for x in blocks if x > 0]) )

{0j, (444-251j), (-307-1615j), (-6396-3404j), (-799-1493j), (634-93j), (-1272-1165j), (917-311j), (-996-2873j), (243-1600j), (-6865-5865j), (1124+481j), (-761-1493j), (-2787-2502j), (-1664-1105j), (-6529-5243j), (1280-163j), (-7892-6193j), (-7107-6518j), (-266-858j), (-307-1866j), (-141-982j), (-7281-6171j), (913+42j), (341-1600j), (1128+421j), (-1664-1165j), (-198-858j), (370-1600j), (-7726-6171j), (-2875-1857j), (-53-982j), (-6218-3404j), (-6759-5421j), (-753-1372j), (844-636j), (-6551-6761j), (-8622-6875j), (887+281j), (-6865-6114j), (266+0j), (-573-2792j), (-266-527j), (917-636j), (-266-389j), (-428-1166j), (634-112j), (151-1037j), (-6419-2857j), (-6885-6518j), (-198-527j), (266-251j), (-307-1679j), (-6759-5243j), (-6511-6072j), (-198-389j), (423-1552j), (-799-1419j), (-1-2218j), (-640+0j), (913+0j), (-8622-7018j), (-428-1493j), (-2522-2096j), (150-982j), (887+421j), (-761-1419j), (-6396-2932j), (444+0j), (-6268-3224j), (423-1337j), (-1272-1105j), (151-1149j), (844-311j), (-6396-34

--- Part Two ---

It turns out that this circuit is very timing-sensitive; you actually need to minimize the signal delay.

To do this, calculate the number of steps each wire takes to reach each intersection; choose the intersection where the sum of both wires' steps is lowest. If a wire visits a position on the grid multiple times, use the steps value from the first time it visits that position when calculating the total value of a specific intersection.

The number of steps a wire takes is the total number of grid squares the wire has entered to get to that location, including the intersection being considered. Again consider the example from above:

    ...........
    .+-----+...
    .|.....|...
    .|..+--X-+.
    .|..|..|.|.
    .|.-X--+.|.
    .|..|....|.
    .|.......|.
    .o-------+.
    ...........

In the above example, the intersection closest to the central port is reached after 8+5+5+2 = 20 steps by the first wire and 7+6+4+3 = 20 steps by the second wire for a total of 20+20 = 40 steps.

However, the top-right intersection is better: the first wire takes only 8+5+2 = 15 and the second wire takes only 7+6+2 = 15, a total of 15+15 = 30 steps.

Here are the best steps for the extra examples from above:

    R75,D30,R83,U83,L12,D49,R71,U7,L72
    U62,R66,U55,R34,D71,R55,D58,R83 = 610 steps
    R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
    U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = 410 steps

What is the fewest combined steps the wires must take to reach an intersection?


In [5]:
from collections import defaultdict

def wander_with_breadcrumbs(path):
    '''Fill in the steps taken on a journey and return a set of visited coordinates with the order visited.'''
    # So now we have to use a dict that uses the position as a key, and then the number of steps it took
    # to reach that point. 
    # TODO: make each dict value a set of integers, just in case a path crosses itself, but not just yet.
    
    direction={
        'R': +1,
        'L': -1,
        'U': +1j,
        'D': -1j
    }
    journey = defaultdict(list)
    
    pos = 0+0j
    steps = 0
    for vector in path:
        facing,dist = vector[0],int(vector[1:])
        #print(facing,dist)
        for n in range(0,dist):
            journey[pos + n*direction[facing]].append(steps)
            steps +=1
        pos += dist*direction[facing]
    return journey

In [6]:
#path = [p.split(',') for p in [l.strip() for l in sample_1.split('\n')] ]
path = [p.split(',') for p in input_03]

j1 = wander_with_breadcrumbs(path[0])
j2 = wander_with_breadcrumbs(path[1])

#print(j1)
crossings = list( set(j1.keys()).intersection( set(j2.keys())) )
#print([(x,j1[x]) for x in crossings])
#print([(x,j2[x]) for x in crossings])

def blocks(x):
    return abs(x.real) + abs(x.imag)


with_dist = [(x, blocks(x), min(j1[x]) + min(j2[x])) for x in crossings]
#print(blocks)
#print(min( [x for x in blocks if x > 0]) )
#print(with_dist)
print(min( [z for x,y,z in with_dist if z>0]) )

19242
